In [1]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Create CLIENT_IT and SECRET_KEY at https://www.reddit.com/prefs/apps  
Click on 'create another app' 
Type in a name (can be anything)  
Click on 'script'  
Type in redirect uri with "http://localhost:8080"
Create app  

Your CLIENT_ID will be under personal use script  
Your SECRET_KEY will be next to secret


In [2]:
CLIENT_ID = '<Under personal use script>'
SECRET_KEY = '<Next to secret>'

Create a file called pw.txt that contains your password in there  
The gitignore will ignore the pw.txt

In [3]:
with open('pw.txt', 'r') as f:
    pw=f.read()

In [4]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

Add in your username in the username data dictionary

In [5]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': '<Your Reddit Username>',
        'password': pw}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyAPI/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

In [6]:
# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

In [7]:
# add authorization to our headers dictionary
headers['Authorization'] = f'bearer {TOKEN}'

In [8]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
# Get the 100 newest stuff on python subreddit.
res = requests.get('https://oauth.reddit.com/r/python/new', headers=headers, params={'limit':'100'})

List of potential data we can extract

In [9]:
post = res.json()['data']['children'][0]
post['data'].keys()

dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'post_hint', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'sel

In [10]:
df = pd.DataFrame()

In [11]:
for post in res.json()['data']['children']:
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'id': post['kind'] + '_' + post['data']['id']
    }, ignore_index=True)

In [12]:
df

,downs,id,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,t3_qhanlj,1.0,"Hi all,\n\nI just published my first package! ...",Python,First Package,1.0,0.67
1,0.0,t3_qha339,2.0,Discussion of using Python in a professional e...,Python,"Thursday Daily Thread: Python Careers, Courses...",2.0,1.00
2,0.0,t3_qh6hv5,1.0,"Hello!, \n\nThis program makes it easier to u...",Python,download videos and music off youtube,1.0,1.00
3,0.0,t3_qh68wz,3.0,"Hello everyone,\n\nI've recently discovered th...",Python,Playing with Python `__path__` attribute,3.0,0.80
4,0.0,t3_qh5xla,1.0,&amp;#x200B;\n\nhttps://i.redd.it/txz7mhvkz1w7...,Python,Announcing the Modzy AI Hackathon 2021- build ...,1.0,1.00
...,...,...,...,...,...,...,...,...
95,0.0,t3_qdr2kp,1.0,\n\nI am working on python flask app which is...,Python,What is best way to model web app subscription...,1.0,0.60
96,0.0,t3_qdqbis,137.0,I wanted to [share a hobby project](https://gi...,Python,Lakshmi: Command-line tool for managing your i...,137.0,0.89
97,0.0,t3_qdpx3n,0.0,I wanted to stop using the same similar passw...,Python,Python passwords manager and generator,0.0,0.43
98,0.0,t3_qdo06u,79.0,\n\nhttps://preview.redd.it/0u8k13t9w1v71.pn...,Python,How to colorize old black and white movies [pr...,79.0,0.92


To get the next 100 newest posts  
To query after a certain id add 'after' in the params  
e.g. t3_qdnyrf (The last id from the previous API call)

In [13]:
# Get the 100 newest stuff on python subreddit.
res = requests.get('https://oauth.reddit.com/r/python/new', headers=headers, params={'limit':'100', 'after': 't3_qdnyrf'})

In [14]:
for post in res.json()['data']['children']:
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'id': post['kind'] + '_' + post['data']['id']
    }, ignore_index=True)

In [15]:
df

,downs,id,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,t3_qhanlj,1.0,"Hi all,\n\nI just published my first package! ...",Python,First Package,1.0,0.67
1,0.0,t3_qha339,2.0,Discussion of using Python in a professional e...,Python,"Thursday Daily Thread: Python Careers, Courses...",2.0,1.00
2,0.0,t3_qh6hv5,1.0,"Hello!, \n\nThis program makes it easier to u...",Python,download videos and music off youtube,1.0,1.00
3,0.0,t3_qh68wz,3.0,"Hello everyone,\n\nI've recently discovered th...",Python,Playing with Python `__path__` attribute,3.0,0.80
4,0.0,t3_qh5xla,1.0,&amp;#x200B;\n\nhttps://i.redd.it/txz7mhvkz1w7...,Python,Announcing the Modzy AI Hackathon 2021- build ...,1.0,1.00
...,...,...,...,...,...,...,...,...
195,0.0,t3_qa2lzi,59.0,,Python,PyPy 7.3.6 release: now with python3.8,59.0,0.94
196,0.0,t3_qa2k9s,3.0,It's been three months since I started with py...,Python,Flash Card Script,3.0,0.80
197,0.0,t3_qa1wev,3.0,,Python,Simple java to python converter,3.0,0.71
198,0.0,t3_qa1i3j,6.0,[Prisma Client Python](https://github.com/Robe...,Python,Prisma Client Python - Type safe database access,6.0,0.88
